In [1]:
import numpy as np
import pandas as pd

In [177]:
col_names = ['sepal_length', 'sepal_width', 'petal_length','petal_Width','type']
data = pd.read_csv('./data/iris.csv',skiprows=1, header=None, names=col_names)
data_classification.head(10)

,sepal_length,sepal_width,petal_length,petal_Width,type
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
5,5.4,3.9,1.7,0.4,Setosa
6,4.6,3.4,1.4,0.3,Setosa
7,5.0,3.4,1.5,0.2,Setosa
8,4.4,2.9,1.4,0.2,Setosa
9,4.9,3.1,1.5,0.1,Setosa


# Decison 

In a decision tree, there are two ypes of nodes, decision nodes and leaf nodes. In leaf nodes we try to have pure classes. In decision nodes we do decide if we go to the left or right tree depending on the condition feature selection algorithm we choose (feature_index and threshold for this feature). We try to maximize the information gain with each split. 
It is a greddy search algortihm, so we try on each solit to mazimeze the decision but don't go back.

## Classification Decision Tree

The tree class for classification problems: (we will not submit this is only for me)

In [206]:
class Node():
    def __init__(self, feature_index=None, threshold=None, leftChild=None, rightChild=None, info_gain=None, value=None):

        self.feature_index = feature_index
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.info_gain = info_gain
        self.threshold = threshold
        self.value = value
        

In [214]:
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2):

        self.root = None

        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+ str(tree.feature_index), "<=", tree.threshold, "? info_gain: ", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.leftChild, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.rightChild, indent + indent)
            
    def build_tree(self, dataset, curr_depth=0):
        train,test = dataset[:,:-1], dataset[:,-1]

        num_samples, num_features = np.shape(train)

        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["infoGain"] > 0:
                leftTree = self.build_tree(best_split['dataset_left'], curr_depth+1)
                rightTree = self.build_tree(best_split['dataset_right'], curr_depth+1)
                return Node(best_split['feature_index'], best_split['threshold'],
                           leftTree, rightTree, best_split['infoGain'])

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):

        best_split={}
        max_info_gain=-float("inf")

        # loop over all features
        for feature_index in range(num_features):
            feature_values=dataset[:, feature_index]
            possible_featurevalues=np.unique(feature_values)

            for threshold in possible_featurevalues:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:,-1], dataset_left[:,-1], dataset_right[:,-1]
                    curr_info_gain=self.information_gain(y, left_y,right_y,"gini")
                    if curr_info_gain > max_info_gain:
                        best_split['feature_index'] = feature_index
                        best_split['threshold'] = threshold
                        best_split['infoGain'] = curr_info_gain
                        best_split['dataset_left'] = dataset_left
                        best_split['dataset_right'] = dataset_right
                        max_info_gain=curr_info_gain

        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode='entrophy'):
        weigth_l = len(l_child)/len(parent)
        weigth_r = len(r_child)/len(parent)
        if mode=='gini':
            gain = self.gini_index(parent) - (weigth_l *self.gini_index(l_child)- weigth_r *self.gini_index(r_child))
        else:
            gain = self.entrophy(parent) - (weigth_l *self.gini_index(l_child)- weigth_r *self.gini_index(r_child))
        return gain


    def entrophy(self, dataset, feature_index, threshold):
        class_labels = np.unique(y)
        entrophy = 0
        for cls in class_labels:
            p_cls = len(y[y==cls])/len(y)
            entrophy += -p_cls - np.log1(p_cls)
        return 1 - entrophy
        
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y==cls])/len(y)
            gini += p_cls ** 2
        return 1 - gini

    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)

    def fit(self, X,Y):
        dataset = np.concatenate((X,Y), axis = 1)
        self.root = self.build_tree(dataset)

    def predict(self, x):
        predictions = [self.make_prediction(x,self.root) for x in x]
        return predictions

    def make_prediction(self, x, tree):
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.leftChild)
        else:
            return self.make_prediction(x, tree.rightChild)

In [215]:
X = data_classification.iloc[:, :-1].values
Y = data_classification.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [216]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_0 <= 4.3 ? info_gain:  1.327592203548086
 left:['Setosa']
 right:X_2 <= 1.0 ? info_gain:  1.3276365381646178
  left:['Setosa']
  right:X_1 <= 2.0 ? info_gain:  1.3274908723055807
    left:['Setosa']
    right:X_1 <= 2.2 ? info_gain:  1.327145379753691
        left:['Setosa']
        right:['Setosa']


In [219]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(Y_test, Y_pred)
print(confusion_matrix(Y_test, Y_pred))

[[ 9  0  0]
 [11  0  0]
 [10  0  0]]


## Regression Decision Tree

In [352]:
class Node():
    def __init__(self, currFeatureIndex=None, threshold=None, leftChild=None, rightChild=None, variance_reduction=None, value=None):

        self.currFeatureIndex = currFeatureIndex
        self.threshold = threshold
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.variance_reduction = variance_reduction

        self.value = value

If it is a regression problem we use variance reduction as a measurement of impurity of a node. 
$\frac{1}{n} \sum_{i=1}^{n} (y_i - \bar{y})^2$
1. Calculate the difference between each data point yi and the mean y.
2. Square each of these differences.
3. Sum up all the squared differences.
4. Divide the sum by the total number of data points n.

In [381]:
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=2):

        self.root = None

        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.currFeatureIndex), "<=", tree.threshold, "?", tree.variance_reduction)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.leftChild, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.rightChild, indent + indent)
            
    def build_tree(self, dataset, curr_depth=0):
        print('build_tree')
        train,test = dataset[:,:-1], dataset[:,-1]

        num_samples, num_features = np.shape(train)

        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            
            best_split = self.get_best_split(dataset, num_samples, num_features)
            print(best_split.keys)
            if len(best_split) != 0 and best_split['variance_reduction'] > 0:
                leftTree = self.build_tree(best_split['dataset_left'], curr_depth+1)
                rightTree = self.build_tree(best_split['dataset_right'], curr_depth+1)
                return Node(best_split['currFeatureIndex'], best_split['threshold'],
                           leftTree, rightTree, best_split['variance_reduction'])

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features) -> dict:
        print('get best split')
        best_split={}
        max_variance_reduction=-float("inf")
        # loop over all features
        for currFeatureIndex in range(num_features):
            print('num_features:' + str(num_features) + " currFeatureIndex:" + str(currFeatureIndex))    
            feature_values=dataset[:, currFeatureIndex]
            possible_featurevalues=np.unique(feature_values)

            for threshold in possible_featurevalues:
                dataset_left, dataset_right = self.split(dataset, currFeatureIndex, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    print(best_split.keys())
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_variance_reduction= self.variance_reduction(y, left_y, right_y)
                    if curr_variance_reduction > max_variance_reduction:
                        best_split['currFeatureIndex'] = currFeatureIndex
                        best_split['threshold'] = threshold
                        best_split['variance_reduction'] = curr_variance_reduction
                        best_split['dataset_left'] = dataset_left
                        best_split['dataset_right'] = dataset_right
                        max_variance_reduction=curr_variance_reduction

        return best_split

    def split(self, dataset, currFeatureIndex, threshold):
        dataset_left = np.array([row for row in dataset if row[currFeatureIndex]<=threshold])
        dataset_right = np.array([row for row in dataset if row[currFeatureIndex]>threshold])
        return dataset_left, dataset_right

    def variance_reduction(self, parent, l_child, r_child):        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        print(parent)
        print(l_child)
        print(r_child)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
        
    def calculate_leaf_value(self, Y):
        Y = np.mean(Y)
        return Y

    def fit(self, X,Y):
        dataset = np.concatenate((X,Y), axis = 1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        predictions = [self.make_prediction(X,self.root) for X in X]
        return predictions

    def make_prediction(self, X, tree):
        if tree.value != None:
            return tree.value
            
        feature_val = X[tree.currFeatureIndex]
        if feature_val <= tree.threshold:
            return self.make_prediction(X, tree.leftChild)
        else:
            return self.make_prediction(X, tree.rightChild)

In [382]:
data = pd.read_csv("./data/airfoil_noise_data.csv")
data = data.head(5)

In [383]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)


In [384]:
regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3)
regressor.fit(X_train,Y_train)
regressor.print_tree()

build_tree
get best split
num_features:5 currFeatureIndex:0
dict_keys([])
[125.951 127.461 127.591 126.201]
[126.201]
[125.951 127.461 127.591]
dict_keys(['currFeatureIndex', 'threshold', 'variance_reduction', 'dataset_left', 'dataset_right'])
[125.951 127.461 127.591 126.201]
[125.951 126.201]
[127.461 127.591]
dict_keys(['currFeatureIndex', 'threshold', 'variance_reduction', 'dataset_left', 'dataset_right'])
[125.951 127.461 127.591 126.201]
[125.951 127.591 126.201]
[127.461]
num_features:5 currFeatureIndex:1
num_features:5 currFeatureIndex:2
num_features:5 currFeatureIndex:3
num_features:5 currFeatureIndex:4
<built-in method keys of dict object at 0x000001DC42D8E940>
build_tree
build_tree
X_0 <= 1250.0 ? 0.525625000000002
 left:126.481
 right:126.481


# RandomForest

It is a collection of randomly chosen decision trees. Build new datasets from our origninal dataset. Randomly select rows from the dataset. One row can occur more then 1 time. (bootstrapping)
We also randomly select features. At the end we select the majority voting on the prediction (this is called aggregation)


In [385]:
from collections import Counter
def bootstrap_sample(X,y):
    n_samples = X.shape[0]
    idxs=np.random.choice(n_samples, size=n_samples, replace=True)
    return X[idxs],y[idxs]

def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [386]:
class Random_Forest():
    def __init__(self, num_trees = 15, max_depth = 15, min_samples_split = 2, num_features = None):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.num_features = num_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for i in range(self.num_trees):
            tree = DecisionTreeRegressor(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            X_sample, y_sample = bootstrap_sample(X,y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0 ,1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [389]:
regressor = Random_Forest(min_samples_split=3, max_depth=3)
regressor.fit(X_train,Y_train)
Y_pred = regressor.predict(X_test)
print('Y_pred')
print(Y_pred)
#print(accuracy_score(Y_test, Y_pred))

build_tree
get best split
num_features:5 currFeatureIndex:0
dict_keys([])
[126.201 127.461 127.461 125.951]
[126.201]
[127.461 127.461 125.951]
dict_keys(['currFeatureIndex', 'threshold', 'variance_reduction', 'dataset_left', 'dataset_right'])
[126.201 127.461 127.461 125.951]
[126.201 125.951]
[127.461 127.461]
num_features:5 currFeatureIndex:1
num_features:5 currFeatureIndex:2
num_features:5 currFeatureIndex:3
num_features:5 currFeatureIndex:4
<built-in method keys of dict object at 0x000001DC42D9D600>
build_tree
build_tree
build_tree
get best split
num_features:5 currFeatureIndex:0
dict_keys([])
[125.951 125.951 127.461 127.591]
[125.951 125.951]
[127.461 127.591]
dict_keys(['currFeatureIndex', 'threshold', 'variance_reduction', 'dataset_left', 'dataset_right'])
[125.951 125.951 127.461 127.591]
[125.951 125.951 127.591]
[127.461]
num_features:5 currFeatureIndex:1
num_features:5 currFeatureIndex:2
num_features:5 currFeatureIndex:3
num_features:5 currFeatureIndex:4
<built-in method k